In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe

from helper_fn import * 

In [2]:
system = dy.enter_system()

# the diagram

yield_event2 = dy.signal_periodic_impulse(period=4, phase=2)


with dy.sub_loop( max_iterations=1000 ) as system:

    cnt = dy.counter()
    system.set_outputs([ cnt ])
  

    # execute the counter above until yield_event. Theb continue the main system for one time-instant
    # and continue inside this subsystem again.
    yield_event = dy.signal_periodic_impulse(period=4, phase=2)
    system.loop_yield( yield_event )


cnt = system.outputs[0]


# define output(s)
dy.append_primay_ouput(cnt, 'cnt')
dy.append_primay_ouput(yield_event2, 'yield_event2')


# generate code for Web Assembly (wasm), requires emcc (emscripten) to build
code_gen_results = dy.generate_code(template=dy.TargetWasm(), folder="generated/sub_loop", build=False)

#
dy.clear()

compiling system Subsystem1000 (level 1)... 
determining the computation order...
building execution paths...
All dependencies are resolved.
compiling system simulation (level 0)... 
determining the computation order...
building execution paths...
All dependencies are resolved.
Generated code will be written to generated/sub_loop .


In [3]:
compiled_system = dyexe.CompiledCode(code_gen_results)
sim_results = dyexe.run_batch_simulation(dyexe.SystemInstance(compiled_system), input_data={}, N=30)

In [4]:
sim_results['cnt']

array([  2.,   6.,  10.,  14.,  18.,  22.,  26.,  30.,  34.,  38.,  42.,
        46.,  50.,  54.,  58.,  62.,  66.,  70.,  74.,  78.,  82.,  86.,
        90.,  94.,  98., 102., 106., 110., 114., 118.])

In [5]:
# assert_equal( sim_results['cnt'] , [ ] )

In [6]:
sim_results['yield_event2']

array([0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])